# making an aggregate master dataframe for baseline model
this notebook is going to make a single standardized long format dataframe for each of my cleaned clinical variable dataframes.

will save this into a file with a cleaned_merged_df prefix.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from pathlib import Path
import seaborn as sns
import numpy as np
import glob

from sklearn.externals.joblib import Memory
memory = Memory(cachedir='/tmp', verbose=0)
#@memory.cache above any def fxn.

%matplotlib inline
plt.style.use('ggplot')

from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('livereveal', {
        'width': 1024,
        'height': 768,
        'scroll': True,
})

%load_ext autotime

/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: The 'cachedir' parameter has been deprecated in version 0.12 and will be removed in version 0.14.
You provided "cachedir='/tmp'", use "location='/tmp'" instead.
  # Remove the CWD from sys.path while we load stuff.


In [2]:
#patients of interest from rotation_cohort_generation
from parameters import final_pt_df_v, date, repository_path

#patients of interest from rotation_cohort_generation
final_pt_df2 = final_pt_df_v #pd.read_csv('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_final_pt_df2.csv'%(most_updated_patient_df), index_col=0)
del(final_pt_df_v)

patients= list(final_pt_df2['subject_id'].unique())
hadm_id= list(final_pt_df2['hadm_id'].unique())
icustay_id= list(final_pt_df2['icustay_id'].unique())
icustay_id= [int(x) for x in icustay_id]


time: 1.22 s


In [3]:
final_pt_df2['icustay_id'].nunique() #14478

19633

time: 4.96 ms


In [4]:
from parameters import lower_window, upper_window, folder, date, time_col, time_var, patient_df

time: 753 µs


In [5]:
#import all cleaned or 'prepped' clinical variable csv
allFiles = glob.glob(str(repository_path)+ '/data/cleaned/%s/'%(folder) + "{}_*.csv".format(date))
allFiles

['/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-opensource/data/cleaned/48_hr_window/30102019_vasopressin_prepped.csv',
 '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-opensource/data/cleaned/48_hr_window/30102019_weight_prepped.csv',
 '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-opensource/data/cleaned/48_hr_window/30102019_gcs_prepped.csv',
 '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-opensource/data/cleaned/48_hr_window/30102019_epinephrine_prepped.csv',
 '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-opensource/data/cleaned/48_hr_window/30102019_leuk_prepped.csv',
 '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-opensource/data/cleaned/48_hr_window/30102019_height_prepped.csv',
 '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-opensource/data/cleaned/48_hr_window/30102019_labs_prepped.csv',
 '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-opensource/data/cleaned/48_hr_window/30102019_bg_ART_prepped.csv',
 '/Us

time: 3.11 ms


In [6]:
save_path= str(repository_path)+'/data/cleaned_merged/'
def save_df(df, df_name='default', save_path=save_path, add_subfolder=False):
    #uses the date and supplied df name and saves to the savepath specified above.
    if df_name == 'default':
        df_name= "%s"%(df)
    
    address=save_path+'%s/'%(folder)
    if not os.path.exists(address):
        print(address)
        os.makedirs(address)
    pd.DataFrame(df).to_csv(Path(address+'%s_%s_cleaned_merged.csv' %(date, df_name)))

time: 6.91 ms


In [7]:
os.getcwd()

'/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-opensource/notebooks'

time: 2.22 ms


In [8]:
#making a dictionary of all my dataframes for easier cycling through

df_list=[]
for element in allFiles:
    df_list.append(element.split('{}_'.format(date))[1].split('_prepped.csv')[0]) #making an list of all my dataframes in order they appear in file

dfs = {}
i=0
for name in df_list:
    dfs[name] = pd.read_csv(allFiles[i],  index_col=0)
    i+=1

/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


time: 12.6 s


In [9]:
#all of the column names 
for element in df_list:
    print(element,':',list(dfs[element]))

vasopressin : ['icustay_id', 't_0', 'value', 'label', 'uom', 'delta']
weight : ['icustay_id', 'value', 'uom', 'label', 'delta', 't_0']
gcs : ['subject_id', 'hadm_id', 'icustay_id', 'delta', 'label', 'value', 't_0', 'uom']
epinephrine : ['icustay_id', 't_0', 'value', 'label', 'uom', 'delta']
leuk : ['hadm_id', 'icustay_id', 'subject_id', 't_0', 'delta', 'value', 'label', 'uom']
height : ['icustay_id', 'value', 'uom', 'label', 'delta', 't_0']
labs : ['delta', 'hadm_id', 'icustay_id', 'label', 'subject_id', 't_0', 'uom', 'value']
bg_ART : ['delta', 'hadm_id', 'icustay_id', 'label', 'subject_id', 't_0', 'uom', 'value']
o2_flow : ['icustay_id', 't_0', 'value', 'label', 'uom', 'delta']
rrt : ['icustay_id', 't_0', 'value', 'label', 'uom', 'delta']
ventcategory : ['icustay_id', 'delta', 'value', 'uom', 't_0', 'label']
sum_elix : ['subject_id', 'hadm_id', 'icustay_id', 'value', 'label', 'delta', 'uom', 't_0']
bg_all : ['subject_id', 'hadm_id', 'icustay_id', 'charttime', 'delta', 't_0', 'label',

## standardizing columns

#### adding icustay_id, dropping hadm_id

In [10]:
##dropping hadm_id from all:
list1=[]
for element in df_list:
    if 'hadm_id' in (list(dfs[element])):
        list1.append(element)

for element in list1:
    dfs[element]= dfs[element].drop('hadm_id', axis=1)

time: 621 ms


In [11]:
##dropping subject_id from all:
list1=[]
for element in df_list:
    if 'subject_id' in (list(dfs[element])):
        list1.append(element)

for element in list1:
    dfs[element]= dfs[element].drop('subject_id', axis=1)

time: 455 ms


In [12]:
#all of the column names 
for element in df_list:
    print(element,':',sorted(list(dfs[element])))

vasopressin : ['delta', 'icustay_id', 'label', 't_0', 'uom', 'value']
weight : ['delta', 'icustay_id', 'label', 't_0', 'uom', 'value']
gcs : ['delta', 'icustay_id', 'label', 't_0', 'uom', 'value']
epinephrine : ['delta', 'icustay_id', 'label', 't_0', 'uom', 'value']
leuk : ['delta', 'icustay_id', 'label', 't_0', 'uom', 'value']
height : ['delta', 'icustay_id', 'label', 't_0', 'uom', 'value']
labs : ['delta', 'icustay_id', 'label', 't_0', 'uom', 'value']
bg_ART : ['delta', 'icustay_id', 'label', 't_0', 'uom', 'value']
o2_flow : ['delta', 'icustay_id', 'label', 't_0', 'uom', 'value']
rrt : ['delta', 'icustay_id', 'label', 't_0', 'uom', 'value']
ventcategory : ['delta', 'icustay_id', 'label', 't_0', 'uom', 'value']
sum_elix : ['delta', 'icustay_id', 'label', 't_0', 'uom', 'value']
bg_all : ['charttime', 'delta', 'icustay_id', 'label', 't_0', 'uom', 'valuenum']
norepinephrine : ['delta', 'icustay_id', 'label', 't_0', 'uom', 'value']
pt_info : ['delta', 'icustay_id', 'label', 't_0', 'uom', 

In [13]:
#dropping charttime, endtime and first_charttime
list1=[]
list2=[]
for element in df_list:
    if 'charttime' in (list(dfs[element])):
        list1.append(element)
    if 'endtime' in (list(dfs[element])):
        list2.append(element)

for element in list1:
    dfs[element]= dfs[element].drop('charttime', axis=1)

for element in list2:
    dfs[element]= dfs[element].drop('endtime', axis=1)
    
#dfs['rrt']= dfs['rrt'].drop('first_charttime', axis=1)

time: 455 ms


In [14]:
#converting valuenum and value to same label
list1=[]

for element in df_list:
    if 'valuenum' in (list(dfs[element])):
        list1.append(element)

for element in list1:
    dfs[element]= dfs[element].rename(index=str, columns={'valuenum':'value'})


time: 164 ms


In [15]:
del(list1,list2)

time: 651 µs


In [16]:
def label_lower(df_name):
    dfs[df_name]['label']= dfs[df_name]['label'].apply(lambda x: x.lower())

time: 1.61 ms


In [17]:
#turning all labels to lowercase
for element in df_list:
    label_lower(element)


time: 2.51 s


In [18]:
#adding a df source table label to each df. 
for element in df_list:
    dfs[element]['source']=element

#adding a patient id to each

for element in df_list:
    dfs[element]= pd.merge(dfs[element], final_pt_df2[['icustay_id','subject_id']], how='left')
    

time: 2.94 s


In [19]:
#all of the column names 
for element in df_list:
    print(element,':',sorted(list(dfs[element])))

vasopressin : ['delta', 'icustay_id', 'label', 'source', 'subject_id', 't_0', 'uom', 'value']
weight : ['delta', 'icustay_id', 'label', 'source', 'subject_id', 't_0', 'uom', 'value']
gcs : ['delta', 'icustay_id', 'label', 'source', 'subject_id', 't_0', 'uom', 'value']
epinephrine : ['delta', 'icustay_id', 'label', 'source', 'subject_id', 't_0', 'uom', 'value']
leuk : ['delta', 'icustay_id', 'label', 'source', 'subject_id', 't_0', 'uom', 'value']
height : ['delta', 'icustay_id', 'label', 'source', 'subject_id', 't_0', 'uom', 'value']
labs : ['delta', 'icustay_id', 'label', 'source', 'subject_id', 't_0', 'uom', 'value']
bg_ART : ['delta', 'icustay_id', 'label', 'source', 'subject_id', 't_0', 'uom', 'value']
o2_flow : ['delta', 'icustay_id', 'label', 'source', 'subject_id', 't_0', 'uom', 'value']
rrt : ['delta', 'icustay_id', 'label', 'source', 'subject_id', 't_0', 'uom', 'value']
ventcategory : ['delta', 'icustay_id', 'label', 'source', 'subject_id', 't_0', 'uom', 'value']
sum_elix : ['d

In [20]:
for element in df_list:
    print(element,'column that dont belong:', [x for x in list(dfs[element]) if x not in ['delta', 'icustay_id', 'label', 'source', 'subject_id', 't_0', 'uom', 'value']])
    print(element,'column that are missing:',[x for x in ['delta', 'icustay_id', 'label', 'source', 'subject_id', 't_0', 'uom', 'value'] if x not in list(dfs[element])])

vasopressin column that dont belong: []
vasopressin column that are missing: []
weight column that dont belong: []
weight column that are missing: []
gcs column that dont belong: []
gcs column that are missing: []
epinephrine column that dont belong: []
epinephrine column that are missing: []
leuk column that dont belong: []
leuk column that are missing: []
height column that dont belong: []
height column that are missing: []
labs column that dont belong: []
labs column that are missing: []
bg_ART column that dont belong: []
bg_ART column that are missing: []
o2_flow column that dont belong: []
o2_flow column that are missing: []
rrt column that dont belong: []
rrt column that are missing: []
ventcategory column that dont belong: []
ventcategory column that are missing: []
sum_elix column that dont belong: []
sum_elix column that are missing: []
bg_all column that dont belong: []
bg_all column that are missing: []
norepinephrine column that dont belong: []
norepinephrine column that ar

In [21]:
pd.to_timedelta(dfs['norepinephrine']['delta']).describe()

count                     42234
mean     0 days 13:27:49.083676
std      0 days 14:44:31.196249
min             0 days 00:00:00
25%             0 days 00:00:00
50%             0 days 09:00:00
75%             1 days 00:30:00
max             2 days 00:00:00
Name: delta, dtype: object

time: 149 ms


In [22]:
pd.to_timedelta(dfs['norepinephrine']['delta']).describe()

count                     42234
mean     0 days 13:27:49.083676
std      0 days 14:44:31.196249
min             0 days 00:00:00
25%             0 days 00:00:00
50%             0 days 09:00:00
75%             1 days 00:30:00
max             2 days 00:00:00
Name: delta, dtype: object

time: 141 ms


# converting data formats

### looking at measured values (quick qc)

In [23]:
len(df_list)

23

time: 2.38 ms


In [24]:
def value_viewer(df_name):
    return(dfs[df_name]['label'].unique())

time: 814 µs


In [25]:
value_viewer('bg_ART')

array(['pco2'], dtype=object)

time: 2.68 ms


In [26]:
value_viewer('bg_all')

array(['calcium', 'lactate', 'ph', 'potassium', 'temperature', 'glucose',
       'hemoglobin', 'chloride', 'sodium', 'bicarbonate'], dtype=object)

time: 30.1 ms


In [27]:
#value_viewer('uti')

time: 447 µs


In [28]:
value_viewer('labs')


array(['bicarbonate', 'bilirubin', 'chloride', 'creatinine', 'glucose',
       'potassium', 'sodium', 'bun', 'hemoglobin', 'platelet', 'wbc',
       'inr', 'ptt', 'lactate', 'bands'], dtype=object)

time: 63.1 ms


In [29]:
value_viewer('vitals')

array(['meanartpress', 'sysbp', 'heartrate', 'resprate', 'spo2',
       'temperature', 'glucose', 'diasbp'], dtype=object)

time: 300 ms


In [30]:
value_viewer('pt_info')

array(['yearsold', 'gender', 'ethnicity'], dtype=object)

time: 4.23 ms


In [31]:
list(dfs)

['vasopressin',
 'weight',
 'gcs',
 'epinephrine',
 'leuk',
 'height',
 'labs',
 'bg_ART',
 'o2_flow',
 'rrt',
 'ventcategory',
 'sum_elix',
 'bg_all',
 'norepinephrine',
 'pt_info',
 'dopamine',
 'phenylephrine',
 'cancer_elix',
 'sofa',
 'pafaRatio',
 'nit',
 'dobutamine',
 'vitals']

time: 1.93 ms


# combining data

In [32]:
set(value_viewer('labs')) & set(value_viewer('bg_all'))

{'bicarbonate',
 'chloride',
 'glucose',
 'hemoglobin',
 'lactate',
 'potassium',
 'sodium'}

time: 55.7 ms


In [33]:
set(value_viewer('labs')) & set(value_viewer('vitals'))

{'glucose'}

time: 331 ms


In [34]:
set(value_viewer('bg_all')) & set(value_viewer('vitals'))

{'glucose', 'temperature'}

time: 307 ms


# combining all df

In [35]:
# making one big dataframe via pd. concat
big_df= pd.concat(dfs.values(), sort=False).sort_values(['icustay_id','delta','label','source'], ascending=True)
#converting delta to time delta, to datetime rounded to 2 minutes, and back to time delta (more efficient than rounding timedeltas)
big_df['delta']= pd.to_timedelta(big_df['delta'])
big_df['delta']= pd.to_datetime(big_df['delta']).dt.round('2min')
big_df['delta']= pd.to_timedelta(big_df['delta'])

/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2910: FutureWarning: Passing timedelta64-dtype data is deprecated, will raise a TypeError in a future version
  exec(code_obj, self.user_global_ns, self.user_ns)


time: 39.1 s


/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: Passing datetime64-dtype data to TimedeltaIndex is deprecated, will raise a TypeError in a future version
  


In [36]:
len(big_df)

7967260

time: 2.22 ms


In [37]:
len(big_df)

7967260

time: 2.38 ms


In [38]:
big_df.drop_duplicates(subset=['icustay_id','label','value','delta',], keep='last', inplace=True) #7638425  -> 7315304 at 2 min. 

time: 4.03 s


In [39]:
len(big_df)

7635452

time: 2.01 ms


In [40]:
#big_df['sum_elix']

time: 400 µs


In [41]:
big_df.groupby('label')['value'].describe() 

,count,unique,top,freq
label,,,,
bands,19633,3,absent,16546
bicarbonate,62333,96,23,4936
bilirubin,18798,583,0.4,1418
bun,61960,339,14,1727
calcium,33713,193,1.13,1673
cancer_elix,19633,2,0,17212
chloride,69606,144,106,4231
creatinine,62214,287,0.7,4783
daily_sofa,43650,24,1,7816


time: 2.69 s


In [42]:
#big_df.groupby('label')['value'].describe() #14478 icustay_id's

time: 486 µs


In [43]:
big_df['label'].unique()

array(['bands', 'cancer_elix', 'daily_sofa', 'dobutamine', 'dopamine',
       'epinephrine', 'heartrate', 'height', 'norepinephrine', 'o2_flow',
       'pao2fio2ratio', 'pco2', 'phenylephrine', 'resprate', 'spo2',
       'sum_elix', 'vasopressin', 'vent_recieved', 'weight', 'diasbp',
       'meanartpress', 'sysbp', 'rrt', 'bicarbonate', 'bilirubin', 'bun',
       'chloride', 'creatinine', 'glucose', 'hemoglobin', 'inr',
       'platelet', 'potassium', 'ptt', 'sodium', 'wbc', 'temperature',
       'ph', 'leukocyte', 'nitrite', 'lactate', 'ethnicity', 'gender',
       'yearsold', 'calcium', 'mingcs'], dtype=object)

time: 668 ms


In [44]:
big_df[big_df['label']=='leukocyte'].head()

,icustay_id,t_0,value,label,uom,delta,source,subject_id
13546,200001.0,2181-11-26,Neg/Not_tested,leukocyte,pos/neg category,1 days,leuk,55973
9126,200003.0,2199-08-04,Neg/Not_tested,leukocyte,pos/neg category,1 days,leuk,27513
9527,200012.0,2153-12-23,Neg/Not_tested,leukocyte,pos/neg category,1 days,leuk,28448
3120,200014.0,2105-02-17,Neg/Not_tested,leukocyte,pos/neg category,1 days,leuk,9514
7162,200019.0,2178-07-09,Neg/Not_tested,leukocyte,pos/neg category,1 days,leuk,21789


time: 522 ms


## quick qc to protect against old versions

In [45]:
#removing firstpos else neg ssc col if it still exists (qc)
big_df=big_df.loc[:,list(big_df.columns!="first_pos_else_neg_ssc")]

time: 711 ms


In [46]:
save_df(big_df, 'longdf')

/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-opensource/data/cleaned_merged/48_hr_window/
time: 2min 19s


In [47]:
del big_df
del dfs

time: 628 ms


#### making a patient missingness visualization

In [48]:
# #big_agg= big_df.groupby(['icustay_id','label'], as_index=False)['value'].agg(['min'])
# big_agg= big_df.groupby(['icustay_id','label'], as_index=False)['value'].size()
# big_agg_count= big_agg.reset_index().pivot(index='icustay_id',columns='label', values=0)#, levels='icustay_id')

time: 1.11 ms


In [49]:
# big_agg_count= big_agg.reset_index().pivot(index='icustay_id',columns='label', values=0)#, levels='icustay_id')

time: 818 µs


In [50]:
# sns.set(rc={'figure.figsize':(25,15)})

time: 390 µs


In [51]:
# #big_agg_min

# #%matplotlib inline
# sns.set(rc={'figure.figsize':(25,15)})
# big_agg_count= big_agg_count.fillna(0)
# big_agg_count = big_agg_count[big_agg_count.columns].astype(float) 


# sns.heatmap(big_agg_count,vmin=0, vmax=1, cmap=sns.color_palette("RdBu_r", 5))

time: 1.14 ms


In [52]:
# #deleting df's for memory once saved
# del big_df, big_agg_count, big_agg, dfs

time: 615 µs


In [53]:
# big_agg_count[big_agg_count>0] =1
# big_agg_pt_missing= big_agg_count.T.apply(lambda x:100*(len(list(big_agg_count))-sum(x))/len(list(big_agg_count)))
# big_agg_pt_missing= pd.DataFrame(big_agg_pt_missing).rename(index=str, columns={0:'%_of_values_missing'})

time: 662 µs


In [54]:
# big_agg_pt_missing.sort_values('%_of_values_missing',ascending=False).plot()

time: 658 µs
